In [1]:
import numpy as np

In [2]:
class Array:
    
    def __init__(self, a):
        self.array = np.array(a, dtype = 'float')
        self.shape = self.shape_f()

    def shape_f(self):
        return self.array.shape
    
    def T(self):
        res = np.zeros((self.shape[1], self.shape[0]))
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[j][i] = self.array[i][j]
        return Array(res)
        

    def plus(self, other):
        res = np.array(self.array)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[i][j] += other.array[i][j]
        return Array(res)

    def pointwise_multiply(self, other):
        res = np.array(self.array)
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                res[i][j] *= other.array[i][j]
        return Array(res)
    

    def matrix_multiply(self, other):
        res = np.zeros((self.shape[0], other.shape[1]))
        for i in range(self.shape[0]):
            for j in range(other.shape[1]):
                for k in range(len(self.array[i])):
                    res[i][j] += self.array[i][k] * other.array[k][j]
        return Array(res)
    def det(self):
        if self.shape[0] == self.shape[1]:
            a = np.array(self.array)
            res = a[0][0] * a[1][1] * a[2][2] + a[0][1] * a[1][2] * a[2][0] + a[1][0] * a[2][1] * a[0][2] - \
                  (a[0][2] * a[1][1] * a[2][0] + a[0][1] * a[1][0] * a[2][2] + a[1][2] * a[2][1] * a[0][0])
            return res
        else: return 0
    
    # Operators
    def __add__(self, other):
        return self.plus(other)
    
    def __sub__(self, other):
        return self.plus(Array(-(other.array)))
    
    def __mul__(self, other):
        return self.pointwise_multiply(other)
    
    def __matmul__(self, other):
        return self.matrix_multiply(other)
    
    def __getitem__(self, ij):
        (i, j) = ij
        return self.array[i][j]

In [3]:
eps = 10**(-3)
N = 11

In [4]:
# Example
A_t = Array([[3, 1, 1], [1, 5, 1], [1, 1, 7]])
b_t = Array([[5, 7, 9]])

In [5]:
# Example 2
A_t2 = Array([[0, 1, 1], [1, 5, 1], [1, 1, 7]])
b_t2 = Array([[2, 20, 12]])

In [6]:
# Test 0
A0 = Array([[0, 2, 3], [1, 2, 4], [4, 5, 6]])
b0 = Array([[13, 17, 32]])

In [7]:
# Test 1
A1 = Array([[N+2, 1, 1], [1, N+4, 1], [1, 1, N+6]])
b1 = Array([[N+4, N+6, N+8]])

In [8]:
# Test 2
A2 = Array([[-(N+2), 1, 1], [1, -(N+4), 1], [1, 1, -(N+6)]])
b2 = Array([[-(N+4), -(N+6), -(N+8)]])

In [9]:
# Test 3
A3 = Array([[-(N+2), N+3, N+4], [N+5, -(N+4), N+1], [N+4, N+5, -(N+6)]])
b3 = Array([[N+4, N+6, N+8]])

In [10]:
# Test 4
A4 = Array([[N+2, N+1, N+1], [N+1, N+4, N+1], [N+1, N+1, N+1]])
b4 = Array([[N+4, N+6, N+8]])

In [11]:
# A_m = np.array([[1, -1], [-1, 1]])
# b_m = np.array([0, 0])

In [12]:
A = [A_t, A_t2, A0, A1, A2, A3, A4]#, A_m]
b = [b_t, b_t2, b0, b1, b2, b3, b4]#, b_m]

In [13]:
def v_norm_inf(v):
    return max(v)
def v_norm_1(v):
    return sum(abs(v))
def v_norm_2(v):
#     return np.sqrt(v@v.T)
    return np.sqrt((v.T() @ v).array[0][0])

def m_norm_inf(A):
    return max(sum(abs(A)))
def m_norm_1(A):
    return m_norm_inf(A.T)
def m_norm_2(A):
#     return np.sqrt(max(np.linalg.eigh(A.T @ A)[0]))
    M = A.T() @ A
    return np.sqrt(max(np.linalg.eigh(M.array)[0]))
def m_norm_f(A):
    return sum(sum(A*A))
    

m_norms = [m_norm_1, m_norm_2, m_norm_inf, m_norm_f]
v_norms = [v_norm_1, v_norm_2, v_norm_inf]
m_n_norm = 1
v_n_norm = 1

# Метод простой итерации

In [14]:
def simple_iteration(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    b = b.T()
    n = A.shape[0]
    mu = 1/m_norms[m_n_norm](A)
    Mu = Array(np.ones(A.shape) * mu)
    B = Array(np.eye(n)) - Mu*A
    if m_norms[m_n_norm](B)>1:
        b = A.T() @ b
        A = A.T() @ A
        mu = 1/m_norms[m_n_norm](A) 
        Mu = Array(np.ones(A.shape) * mu)
        B = Array(np.eye(n)) - Mu*A
        if m_norms[m_n_norm](B)>1:
            print('Norm B > 1')
            return 0
        
    Mu = Array(np.ones(b.shape) * mu)
    c = Mu * b
    x_old = c
    x = B @ x_old + c
    
    B_norm = m_norms[m_n_norm](B)/(1-m_norms[m_n_norm](B))
    iterations = 1
    while B_norm * v_norms[v_n_norm](x-x_old) > eps:
        x_old = x
        x = B @ x_old + c
        iterations +=1
    print('Iterations:', iterations)
    print('x:', x.T().array, '\n')
    return x

In [15]:
for i in range(len(A)):
    print("\t\tTest", i)
    simple_iteration(A[i], b[i])

		Test 0
Iterations: 15
x: [[0.99912225 1.00030081 1.00012856]] 

		Test 1
Iterations: 6717
x: [[9.99902056e+00 2.00016424e+00 1.11948536e-04]] 

		Test 2
Iterations: 268
x: [[0.99979257 2.00058599 2.99966911]] 

		Test 3
Iterations: 5
x: [[0.99969301 1.00009038 1.00005194]] 

		Test 4
Iterations: 11
x: [[1.35193161 1.30817366 1.27399604]] 

		Test 5
Iterations: 30
x: [[1.29920858 1.11832702 1.08163052]] 

		Test 6
Iterations: 749
x: [[-3.9993869  -0.66653284  6.24922453]] 



# Метод Зейделя

In [16]:
def seidel(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    n = A.shape[0]
    b = b.T()
    
    M = [abs(A[i, i]) > (sum(abs(A.array[i]))-abs(A[i, i])) for i in range(n)]
    if sum(M) != n:
        b = A.T() @ b
        A = A.T() @ A
    C = Array(A.array)
    C = Array([C.array[i]*(-1/A[i, i]) for i in range(n)]) + Array(np.eye(n))
    d = Array([[b[i, 0]/A[i, i] for i in range(n)]]).T()
    x_old = Array(d.array)
    x = Array(x_old.array)
    for i in range(n):
        x_old = Array(x.array)
        x.array[i] = (Array([C.array[i]]) @ x_old)[0, 0] + d[i, 0]
    iterations = 1
    while v_norms[v_n_norm](A @ x - b) > eps:
        for i in range(n):
            x_old = Array(x.array)
            x.array[i] = (Array([C.array[i]]) @ x_old)[0, 0] + d[i, 0]
        iterations += 1
    print('Iterations:', iterations)
    print('x', x.T().array)
    return x

In [17]:
for i in range(len(A)):
    print("\t\tTest", i)
    seidel(A[i], b[i])

		Test 0
Iterations: 4
x [[1.00022736 0.99997049 0.99997174]]
		Test 1
Iterations: 155
x [[9.99205852e+00 2.00130288e+00 9.13616319e-04]]
		Test 2
Iterations: 333
x [[1.00106769 1.99772852 3.00114836]]
		Test 3
Iterations: 3
x [[1.00001096 1.00000029 0.99999934]]
		Test 4
Iterations: 3
x [[1.35248617 1.3084388  1.27417206]]
		Test 5
Iterations: 18
x [[1.29974595 1.1188286  1.08220513]]
		Test 6
Iterations: 8464
x [[-3.99853703 -0.66628931  6.24808758]]


# Метод Гаусса

In [18]:
def gauss(A, b):
    if not A.det() :
        print("Матрица А особая")
        return 0
    A = Array(A.array)
    n = A.shape[0]
    b = b.T()
    for i in range(n):
        T = A.T()
        m = np.argmax(abs(T.array[i][i:]))
        A.array[i], A.array[i+m] = np.array(A.array[i+m]), np.array(A.array[i])
        b.array[i], b.array[i+m] = np.array(b.array[i+m]), np.array(b.array[i])
        a = A[i, i]
        A.array[i] = A.array[i]*(1/a)
        b.array[i] = b.array[i]*(1/a)
        for j in range(i+1, n):
            b.array[j] = np.array(b.array[j] - A[j, i] * b.array[i])
            A.array[j] = np.array(A.array[j] - A[j, i] * A.array[i])
    x = Array(np.zeros(b.shape))
    x.array[n-1] = b.array[n-1]
    for i in range(n-1):
        tmp = Array(-x.array)
        x.array[n-2-i] = (Array([A.array[n-2-i]]) @ tmp)[0,0] + b.array[n-2-i]
    print(x.T().array)
    return x

In [19]:
for i in range(len(A)):
    print("\t\tTest", i)
    gauss(A[i], b[i])

		Test 0
[[1. 1. 1.]]
		Test 1
[[10.  2.  0.]]
		Test 2
[[1. 2. 3.]]
		Test 3
[[1. 1. 1.]]
		Test 4
[[1.35250918 1.30844553 1.27417381]]
		Test 5
[[1.29974851 1.11883056 1.08220685]]
		Test 6
[[-4.         -0.66666667  6.25      ]]
